In [ ]:
import GCN
import GAT
import utils
import torch.nn as nn
from torch.optim import SGD, Adam
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

NCLASS = 7
NITER = 1000
NTEST = 100
LR = 0.001

# Data

In [ ]:
adj, features, labels, idx_train, idx_val, idx_test = utils.load_data()

In [ ]:
plt.imshow(adj[:len(idx_train), :len(idx_train)].cpu(), cmap ='gray')

# Model

In [ ]:
# model = GCN.GCN(features.shape[0], features.shape[1], 1024, NCLASS, 0.3, False).cuda()
model = GAT.GAT(features.shape[1], 16, adj.shape[0], NCLASS, 0.3).cuda()
loss_fn = nn.CrossEntropyLoss()
optim = Adam(model.parameters(), lr=LR)

# Training

In [ ]:
features = features.cuda()
adj = adj.cuda()
labels = labels.cuda()

for i in range(NITER):
    model.train()
    optim.zero_grad()
    outputs = model(features, adj)
    predicts = outputs.argmax(dim=1)
    loss = loss_fn(outputs[idx_train], labels[idx_train])
    loss.backward()
    optim.step()
    
    precision = accuracy_score(labels[idx_train].cpu(), predicts[idx_train].cpu())
    print("[{:3d}/{:3d}]  loss: {:.4f}   precision: {:5.2%}".format(i+1, NITER, loss, precision))
    
    if(i % NTEST == NTEST-1):
        model.eval()
        outputs = model(features, adj)
        predicts = outputs.argmax(dim=1)
        loss = loss_fn(outputs[idx_test], labels[idx_test])
        precision = accuracy_score(labels[idx_test].cpu(), predicts[idx_test].cpu())
        print("=============================================")
        print("[Testing]  loss: {:.4f}   precision: {:5.2%}".format(loss, precision))
        print("=============================================")